In [3]:
import pandas as pd
from ortools.linear_solver import pywraplp
import time

def solve_conservation_model(df_path, budget, alpha=0.1, time_limit_sec=300):
    """
    Implementación del Modelo ILP de Conservación (Menorca).
    
    Args:
        df_path (str): Ruta al CSV enriquecido (final_dataset.csv).
        budget (float): Presupuesto total disponible (B).
        alpha (float): Bonus de conectividad para corredores.
        time_limit_sec (int): Tiempo máximo para el solver.
    """
    print(f"\n--- INICIANDO OPTIMIZACIÓN (Presupuesto: {budget}€) ---")
    
    # 1. CARGA DE DATOS
    # ---------------------------------------------------------
    df = pd.read_csv(df_path)
    df.set_index('grid_id', inplace=True) # Usamos grid_id como índice para acceso rápido
    
    # Definimos Conjuntos (Sets)
    I = df.index.tolist() # Todas las celdas
    S = ['atelerix', 'martes', 'eliomys', 'oryctolagus'] # Especies
    
    # 2. INICIALIZAR SOLVER
    # ---------------------------------------------------------
    # Usamos SCIP (Solving Constraint Integer Programs), estándar para MIP gratuito
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        print("¡Error! Solver SCIP no encontrado.")
        return None

    solver.SetTimeLimit(time_limit_sec * 1000) # Milisegundos

    # 3. VARIABLES DE DECISIÓN
    # ---------------------------------------------------------
    print("Generando variables...")
    
    x = {} # Selección de Hábitat: x[i, s]
    y = {} # Inversión en Adaptación: y[i, s]
    z = {} # Corredores: z[i, j]

    # Generamos x e y
    for i in I:
        for s in S:
            x[i, s] = solver.IntVar(0, 1, f'x_{i}_{s}')
            y[i, s] = solver.IntVar(0, 1, f'y_{i}_{s}')

    # Generamos z (Corredores)
    # IMPORTANTE: Solo creamos variable si i y j son vecinos y para evitar duplicados (i < j)
    # Esto reduce el número de variables a la mitad.
    possible_corridors = []
    
    for i in I:
        neighbors_str = df.loc[i, 'neighbors']
        if pd.isna(neighbors_str): continue
        
        neighbors = neighbors_str.split(',')
        for j in neighbors:
            if j in df.index and i < j: # Orden léxico para unicidad (Edge i-j es igual a j-i)
                z[i, j] = solver.IntVar(0, 1, f'z_{i}_{j}')
                possible_corridors.append((i, j))

    print(f"Variables creadas: {solver.NumVariables()}")

    # 4. RESTRICCIONES (Constraints)
    # ---------------------------------------------------------
    print("Aplicando restricciones...")

    # R1. Consistencia de Hábitat (Logic of Investment)
    # x <= P + y
    for i in I:
        for s in S:
            # Nombre de la columna original: 'has_atelerix_algirus', etc.
            # Mapeamos nombre corto a nombre de columna
            full_name = {
                'atelerix': 'has_atelerix_algirus',
                'martes': 'has_martes_martes',
                'eliomys': 'has_eliomys_quercinus',
                'oryctolagus': 'has_oryctolagus_cuniculus'
            }[s]
            
            p_is = int(df.loc[i, full_name]) # 1 si está presente, 0 si no
            solver.Add(x[i, s] <= p_is + y[i, s])

    # R2. Presupuesto (Budget Constraint)
    # Sum(Cost_Adapt * y) + Sum(Cost_Corr * z) <= B
    total_cost_expr = 0
    
    # Costes de Adaptación
    for i in I:
        for s in S:
            cost_adapt = df.loc[i, f'cost_adaptation_{s}']
            total_cost_expr += cost_adapt * y[i, s]
            
    # Costes de Corredores (Media de ambos extremos)
    for (i, j) in possible_corridors:
        cost_i = df.loc[i, 'cost_corridor']
        cost_j = df.loc[j, 'cost_corridor']
        avg_cost = (cost_i + cost_j) / 2.0
        total_cost_expr += avg_cost * z[i, j]
        
    solver.Add(total_cost_expr <= budget)

    # R3. Incompatibilidad Biológica (Hard Constraints)
    # Solo iteramos sobre las celdas conflictivas (Optimización de código)
    
    # A. Marta vs Lirón (Conflict ME)
    conflict_me = df[df['conflict_martes_eliomys'] == 1].index
    for i in conflict_me:
        solver.Add(x[i, 'martes'] + x[i, 'eliomys'] <= 1)
        
    # B. Marta vs Conejo (Conflict MO)
    conflict_mo = df[df['conflict_martes_oryctolagus'] == 1].index
    for i in conflict_mo:
        solver.Add(x[i, 'martes'] + x[i, 'oryctolagus'] <= 1)

    # R4. Topología de Corredores
    # z_ij <= Sum(x_i)  y  z_ij <= Sum(x_j)
    # Un corredor solo existe si ambos extremos están activos para AL MENOS una especie
    for (i, j) in possible_corridors:
        # Suma de todas las especies en la celda i
        sum_species_i = sum(x[i, s] for s in S)
        sum_species_j = sum(x[j, s] for s in S)
        
        solver.Add(z[i, j] <= sum_species_i)
        solver.Add(z[i, j] <= sum_species_j)

    # 5. FUNCIÓN OBJETIVO
    # ---------------------------------------------------------
    # Maximize Z = Sum(Area * Quality * x) + alpha * Sum(z)
    obj_expr = 0
    
    # Valor del Hábitat
    for i in I:
        area = df.loc[i, 'cell_area_km2']
        for s in S:
            quality = df.loc[i, f'suitability_{s}']
            obj_expr += (area * quality) * x[i, s]
            
    # Bonus de Conectividad
    for (i, j) in possible_corridors:
        obj_expr += alpha * z[i, j]
        
    solver.Maximize(obj_expr)

    # 6. RESOLVER
    # ---------------------------------------------------------
    print(f"Resolviendo modelo con {solver.NumConstraints()} restricciones...")
    status = solver.Solve()

    # 7. RESULTADOS
    # ---------------------------------------------------------
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print(f"¡Solución Encontrada! Estado: {status}")
        print(f"Valor Objetivo (Z): {solver.Objective().Value():.2f}")
        
        # Recopilar resultados en un DataFrame limpio
        results = []
        for i in I:
            row = {'grid_id': i}
            is_active_any = False
            for s in S:
                # Guardamos si la celda es hábitat activo (1) o no (0)
                val = int(x[i, s].solution_value())
                row[f'active_{s}'] = val
                if val > 0: is_active_any = True
                
            if is_active_any: # Solo guardamos celdas activas para ahorrar espacio
                results.append(row)
        
        return pd.DataFrame(results)
    else:
        print("No se encontró solución óptima.")
        return None

In [4]:
dataset_path = "../2_data/processed/final_dataset.csv"
    
    # DEFINICIÓN DE ESCENARIOS PRESUPUESTARIOS
    # Nota: Los costes en el dataset están en miles de euros (k€).
    # Ejemplo: 4.5 en el csv significa 4,500 €.
    
PRESUPUESTO_REAL_EUROS = 2000000  # Ejemplo: 2 Millones de Euros
    
# Conversión a unidades del modelo (k€)
PRESUPUESTO_MODELO = PRESUPUESTO_REAL_EUROS / 1000.0

print(f"Presupuesto Real: {PRESUPUESTO_REAL_EUROS:,.0f} €")
print(f"Presupuesto Modelo: {PRESUPUESTO_MODELO} (unidades k€)")

# Ejecutar optimización
solucion = solve_conservation_model(dataset_path, PRESUPUESTO_MODELO)

if solucion is not None:
    print("\n--- RESUMEN DE LA SOLUCIÓN ---")
    print(solucion.head())
    
    # Guardar
    output_path = f"../5_results/tables/solution_budget_{int(PRESUPUESTO_MODELO)}k.csv"
    solucion.to_csv(output_path, index=False)
    print(f"Solución guardada en: {output_path}")
    
    # Análisis rápido de gasto
    print(f"Celdas activadas: {len(solucion)}")

Presupuesto Real: 2,000,000 €
Presupuesto Modelo: 2000.0 (unidades k€)

--- INICIANDO OPTIMIZACIÓN (Presupuesto: 2000.0€) ---


FileNotFoundError: [Errno 2] No such file or directory: '../2_data/processed/final_dataset.csv'